In [37]:
import pandas as pd

In [38]:
nombre_archivo = 'producto.parquet'
# este nombre tiene que tomarse de la ejecución del script; para desarrollarlo lo hardcodeamos
extension = nombre_archivo.split('.')[1]
# guarda la extension del archivo
ruta = './Datasets/'
# harcodeamos ruta relativa

In [39]:
dfprodu = pd.read_parquet(str(ruta)+str(nombre_archivo))

In [40]:
dfprodu.head()
#primer preview

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un,None,None,None
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un,None,None,None
2,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg,None,None,None
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito 1 Kg,1.0 kg,None,None,None
4,0000000205955,SIN MARCA,Chiquizuela Novillito 1 Kg,1.0 kg,None,None,None


In [41]:
dfprodu.dtypes
#comprobamos tipos de dato de cada col y vemos q no hay datos numericos

id              object
marca           object
nombre          object
presentacion    object
categoria1      object
categoria2      object
categoria3      object
dtype: object

In [42]:
dfprodu.describe()
# entender un poco los datos preliminarmente
# vemos que la columna id tiene todos valores unicos y no faltan datos
# hay 2 faltantes o nulos en marca, nombre y presentacion
# las columnas de categoria contienen solo 4 valores cada una, en mas de 72k registros. 
# estas ultimas no presentan ninguna utilidad para este análisis. Las vamos a descartar.
# si más adelante se consigue la información y se quieren utilizar para algo, siempre será posible agregarlas.

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
count,72038,72036,72036,72036,4,4,4
unique,72038,5653,63896,1168,3,3,4
top,0000000001663,SIN MARCA,Mollejas 1 Kg,1.0 kg,Almacén,Aceites,Acondicionadores
freq,1,8649,26,12894,2,2,1


In [43]:
dfprodu.pop('categoria1')
dfprodu.pop('categoria2')
dfprodu.pop('categoria3')
# quitamos las 3 columnas de categoria

0        None
1        None
2        None
3        None
4        None
         ... 
72033    None
72034    None
72035    None
72036    None
72037    None
Name: categoria3, Length: 72038, dtype: object

In [44]:
#busco los registros sin info de marca, nombre o presentacion; para ver si a partir de los datos podempos inferir y completar el resto
dfprodu[dfprodu['marca'].isnull()|dfprodu['nombre'].isnull()|dfprodu['presentacion'].isnull()]
#observamos que son 2 los registros que están incompletos en las 3 columnas. 
#AL no aportar ningún tipo de información adicional, decidimos eliminar los registros.

,id,marca,nombre,presentacion
53619,7798024299045,None,None,None
55798,7798061190213,None,None,None


In [45]:
#obtengo una lista con los indices de las filas a eliminar
filas_a_eliminar=dfprodu[dfprodu['marca'].isnull()|dfprodu['nombre'].isnull()|dfprodu['presentacion'].isnull()].index.tolist()
# y a continuación las elimino
dfprodu.drop(filas_a_eliminar, inplace=True)

In [46]:
# Analisis columna id
# es la columna más importante para corregir dado que es el id de cada producto.
# De acuerdo a la información provista que acompaña los datasets, esta serie debe tener 13 caracteres numéricos, y debiera corresponder al cod EAN del producto 
# debo verificar entonces que todos los registros esten compuestos de 13 caracteres numéricos.

In [47]:
# verifico cantidad de caracteres
# creo una columna 'cant' que indique la cantidad de caracteres
dfprodu['cant']=dfprodu['id'].apply(lambda x: len(str(x)))

In [48]:
#observo que hay registros con 13, 17 y 18 caracteres
dfprodu['cant'].unique()

array([13, 17, 18], dtype=int64)

In [49]:
# observo cómo son esos registros 'erroneos'; y veo que tienen una codificación que no sirve
dfprodu[dfprodu['cant']==18].head()

,id,marca,nombre,presentacion,cant
2287,10-1-2300006000003,MILKAUT,Queso Sardo Milkaut 1 Kg,1.0 kg,18
2288,10-1-2300049000008,SIN MARCA,Zapallo Ondeone 1 Kg,1.0 kg,18
2289,10-1-2300073000005,SIN MARCA,Zapallo Kuki 1 Kg,1.0 kg,18
2290,10-1-2300075000003,SIN MARCA,Zapallo Tasty 1 Kg,1.0 kg,18
2291,10-1-2300160000000,SIN MARCA,Arroz Azafranado 1 Kg,1.0 kg,18


In [50]:
# me quedo con los ultimos 13 caracteres que son los que contienen la info necesaria para ese campo
dfprodu['id2']=dfprodu['id'].apply(lambda x: x[-13:])

In [51]:
# reviso que no haya caracteres no numericos en la nueva columna usando RegEx
import re


In [52]:
dfprodu['caractid2']=dfprodu['id2'].apply(lambda x: re.findall('\D',x))

In [53]:
dfprodu.head()
#voy observando que el campo caractid2 es una lista vacía

,id,marca,nombre,presentacion,cant,id2,caractid2
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un,13,0000000001663,[]
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un,13,0000000002288,[]
2,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg,13,0000000205870,[]
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito 1 Kg,1.0 kg,13,0000000205894,[]
4,0000000205955,SIN MARCA,Chiquizuela Novillito 1 Kg,1.0 kg,13,0000000205955,[]


In [54]:
dfprodu.pop('cant')

0        13
1        13
2        13
3        13
4        13
         ..
72033    13
72034    13
72035    13
72036    13
72037    13
Name: cant, Length: 72036, dtype: int64

In [55]:
dfprodu.describe()
# veo que tiene todos valores duplicados id2. los debo quitar

,id,marca,nombre,presentacion,id2,caractid2
count,72036,72036,72036,72036,72036,72036
unique,72036,5653,63896,1168,67941,1
top,0000000001663,SIN MARCA,Mollejas 1 Kg,1.0 kg,0000000080001,[]
freq,1,8649,26,12894,5,72036


In [56]:
dfprodu['id']=dfprodu['id2']

In [57]:
dfprodu.pop('id2')
dfprodu.pop('caractid2')

0        []
1        []
2        []
3        []
4        []
         ..
72033    []
72034    []
72035    []
72036    []
72037    []
Name: caractid2, Length: 72036, dtype: object

In [58]:
dfprodu.head()

,id,marca,nombre,presentacion
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un
2,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito 1 Kg,1.0 kg
4,0000000205955,SIN MARCA,Chiquizuela Novillito 1 Kg,1.0 kg


In [59]:
#quito duplicados
dfprodu = dfprodu.drop_duplicates(['id'], keep='first')

In [60]:
dfprodu.describe()

,id,marca,nombre,presentacion
count,67941,67941,67941,67941
unique,67941,5648,63744,1168
top,0000000001663,SIN MARCA,Cebolla 1 Kg,1.0 kg
freq,1,6455,13,9077


In [61]:
def sacar_espacios(df):
    '''
    Quitar espacios vacíos de de ambos lados de las cols con strings.
    '''
    sacar_espacios = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(sacar_espacios)
sacar_espacios(dfprodu)

,id,marca,nombre,presentacion
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un
2,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito 1 Kg,1.0 kg
4,0000000205955,SIN MARCA,Chiquizuela Novillito 1 Kg,1.0 kg
...,...,...,...,...
72033,9569753142128,DELI-SITAS,Milhojas Cobertura de Chocolate Blanco Deli-Si...,500.0 gr
72034,9795403001143,MAYO,Mini Pizzetas Mayo 12 Un,12.0 un
72035,9990385651922,DANA,Te Negro en Hebras Lata Dana 50 Gr,50.0 gr
72036,9990385651939,DANA,Te Verde en Hebras Lata Dana 50 Gr,50.0 gr


In [62]:
dfprodu_check = dfprodu['nombre'].str.rsplit(' ', n=1, expand=True)
#checkeo haciendo split en el ultimo espacio a ver si encuentro patrón para limpiar las descripciones de las unidades de medida.
dfprodu_check[1].unique()
#
dfprodu_check[1].describe()

count     67941
unique      214
top          Gr
freq      27786
Name: 1, dtype: object

In [63]:
dfprodu_check = dfprodu['presentacion'].str.rsplit(' ', n=1, expand=True)
#checkeo haciendo split en el espacio a ver si encuentro patrón para limpiar las descripciones de las presentaciones.
dfprodu_check.head()


,0,1
0,1.0,un
1,1.0,un
2,1.0,kg
3,1.0,kg
4,1.0,kg


In [64]:
dfprodu_check[0]=dfprodu_check[0].astype(float)
dfprodu_check[0].describe()

count    67941.000000
mean       214.973389
std        254.076491
min          0.800000
25%          1.500000
50%        100.000000
75%        375.000000
max       1125.000000
Name: 0, dtype: float64

In [65]:

dfprodu_check[1].describe()

count     67941
unique        7
top          gr
freq      27688
Name: 1, dtype: object

In [66]:
dfprodu_check[1].unique()
#observo que la col [0] tiene todas unidades de medida float, y la columna [1] tiene tipos de unidades de medida.
# avanzo, y agrego esas columnas al dataframe original, en reemplazo de la columna 'presentacion' ya que a futuro me va a permitir
# realizar mejores análisis. ej podré hacer queries sobre productos que se venden fraccionados o por unidad.

array(['un', 'kg', 'gr', 'ml', 'cc', 'lt', 'mt'], dtype=object)

In [67]:
dfprodu['presentacion_cant'] = dfprodu_check[0]
dfprodu['presentacion_UMB'] = dfprodu_check[1]
# UM quiere decir unidad de medida base

In [68]:
dfprodu.pop('presentacion')

0          1.0 un
1          1.0 un
2          1.0 kg
3          1.0 kg
4          1.0 kg
           ...   
72033    500.0 gr
72034     12.0 un
72035     50.0 gr
72036     50.0 gr
72037    150.0 gr
Name: presentacion, Length: 67941, dtype: object

In [69]:
#por ultimo titulo las columnas marca, nombre y 'presentacion_UMB'
def titular(df):
    '''
    Convierte a mayúscula la 1era letra de cada palabra en todas las cols.
    '''
    titular = lambda x: x.title() if isinstance(x, str) else x
    return df.applymap(titular)
dfprodu = titular(dfprodu)


In [70]:
dfprodu.rename(columns={'id': 'Id_Producto'}, inplace=True)

In [71]:
dfprodu.head(10)

,Id_Producto,marca,nombre,presentacion_cant,presentacion_UMB
0,0000000001663,La Anónima,Radicheta Atada La Anonima 1 Un,1.0,Un
1,0000000002288,La Anónima,Perejil Atado La Anonima 1 Un,1.0,Un
2,0000000205870,Sin Marca,Ojo De Bife 1 Kg,1.0,Kg
3,0000000205894,Sin Marca,Milanesa De Peceto Novillito 1 Kg,1.0,Kg
4,0000000205955,Sin Marca,Chiquizuela Novillito 1 Kg,1.0,Kg
5,0000000205979,Sin Marca,Espinazo De Novillito 1 Kg,1.0,Kg
6,0000000206020,Sin Marca,Carnanza Comun De Novillo 1 Kg,1.0,Kg
7,0000000206044,Sin Marca,Falda Deshuesada Novillito Bandeja 1 Kg,1.0,Kg
8,0000000206051,Sin Marca,Carne Picada Comun 1 Kg,1.0,Kg
9,0000000206105,Sin Marca,EntrañA De Novillito 1 Kg,1.0,Kg


In [72]:
#exportacion a csv de la tabla
#from encodings import utf_8


ruta_export = './Data_out/'
nombre_archivo_out = 'producto_out.csv'
dfprodu.to_csv(ruta_export+nombre_archivo_out, index=False, encoding='utf-8-sig')